In [24]:
import selenium
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
import re 
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
import xml.etree.ElementTree as ET
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


# Create a sql connection
conn = sqlite3.connect('lab.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS shops')
cur.execute('''CREATE TABLE IF NOT EXISTS shops( 
id INTEGER PRIMARY KEY, 
Name TEXT UNIQUE, Address TEXT, 
Contact TEXT, Phone TEXT,
Location TEXT, Category TEXT, Website TEXT, 
Email TEXT, Facebook TEXT, Twitter TEXT, 
Instagram TEXT, LinkedIn TEXT,Information TEXT, Genre TEXT); ''')

# Establish a path to the web driver
PATH = '/Users/emilyvincett/Desktop/chromedriver'
driver = webdriver.Chrome(PATH)

# Outline the website to scrape
driver.get('https://www.theunsignedguide.com/members/search/15////')

# Pull the xpath required to input email and send
email = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div/div/div/div/div[2]/div/form/div[1]/div[1]/input')))
email.send_keys('biokporsolomon@yahoo.co.uk')

# Pull the xpath required to input email and send
password = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div/div/div/div/div[2]/div/form/div[1]/div[2]/input')))
password.send_keys('@Aurora24')

# Pull the class name for the signin button and click
signin = driver.find_element_by_class_name('btn.btn-lg.btn-brand')
signin.click()


skip_button = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'introjs-button.introjs-skipbutton')))
skip_button.click()

# Handles the search and selection 
looking_for = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div/div/div/div/div[1]/div/form/div[1]/div')))
looking_for.click()

search_term = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div/div/div/div/div[1]/div/form/div[1]/div/div/ul/li[63]')))
search_term.click()

submit = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div/div/div/div/div[1]/div/form/div[4]/button')))
submit.click()


# Logic to control the search, selection, next item and next page
b = 0
while b < 21:
    a = 0 
    while a < 30:
        try:
            time.sleep(2.5)
            results = driver.find_elements_by_class_name('search-result-item')
            name = WebDriverWait(results[a],10).until(EC.presence_of_element_located((By.TAG_NAME,'h4'))).text
            location = WebDriverWait(results[a],10).until(EC.presence_of_element_located((By.TAG_NAME,'p'))).text
            results[a].click()
        
            try:
                category = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'text-uppercase.ng-binding'))).text
            except TimeoutException:
                pass
      
            try:
                time.sleep(1.5)
                listing = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'listing-body')))
                txt = listing.text.split('\n')
                _ = pd.DataFrame(txt)
            
                try:
                    address = _[0].iloc[0]
                except: 
                    address = 'No address data'
                    
                # Contact
                if len(_[_[0].str.contains('(-).+')])==0:
                    contact = 'No named contact person'
                else:
                    try:
                        contact = _[_[0].str.contains('(-).+')].values[0][0]
                    except:
                        address = 'No address data'
                        pass
                
                # Phone No
                if len(_[_[0].str.contains('(\d{3,5}.?\d{3,6}?.\d{1,4})')])==0:
                    phone = 'No Phone Data'     
                else:
                    try:
                        phone = _[_[0].str.contains('(\d{3,5}.?\d{3,6}?.\d{1,4})')].values[0][0]
                    except:
                        phone = 'No Phone Data'
                        pass
                
                # Website
                if len(_[_[0].str.contains('(https?:[a-zA-Z0-9_.+-/#~]+)')])==0:
                    website = 'No Website Data'   
                else:
                    try:
                        website = _[_[0].str.contains('(https?:[a-zA-Z0-9_.+-/#~]+)')].values[0][0].split(':')[1].lstrip('//')
                    except:
                        website = 'No Website Data'
                                
                # Email
                if len(_[_[0].str.contains('([^@]+@[^@]+\.[^@]+)')])==0:
                    email = 'No Email Address'
                else:
                    try:
                        email = _[_[0].str.contains('([^@]+@[^@]+\.[^@]+)')].values[0][0]
                    except:
                        email = 'No Email Data'
                        pass
            
                # Facebook
                if len(_[_[0].str.contains('(^Face.*)')])==0:
                     facebook = 'No Facebook Page'             
                else:
                    try:
                        facebook = _[_[0].str.contains('(^Face.*)')].values[0][0].split(':')[1]
                    except:
                        facebook = 'No Facebook Page'
                        pass
                        
                
                 # Twitter
                if len(_[_[0].str.contains('(Twitt.*)')])==0:
                     twitter = 'No Twitter Page'                    
                else:
                    try:
                        twitter =_[_[0].str.contains('(Twitt.*)')].values[0][0].split(':')[1]
                    except:
                        twitter = 'No Twitter Data'
                        pass
                
                # Instagram
                if len(_[_[0].str.contains('(Inst.*)')])==0:
                     insta = 'No Instagram Page'              
                else:
                    try:
                        insta = _[_[0].str.contains('(Inst.*)')].values[0][0].split(':')[1]
                    except:
                        insta = 'No Instagram Page'
                     
                # LinkedIn    
                if len(_[_[0].str.contains('Linked.*')])==0:
                     linkedin = 'No LinkedIn Page'     
                else:
                    try:
                        linkedin = _[_[0].str.contains('Linked.*')].values[0][0]
                    except:
                        linkedin = 'No LinkedIn Page'
                        pass
            
                try:
                    info = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'col-md-12.ng-binding'))).text
                except TimeoutException:
                    info = 'None'
                    pass
            
                try:
                    genre = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'col-md-8.ng-binding'))).text
                except TimeoutException:
                    genre = 'All Types Of Music'
                    pass
                 
            
                print(name,address,contact,phone,location,category,website,email,facebook,twitter,insta,linkedin,info,genre)
                cur.execute('INSERT OR IGNORE INTO shops(Name,Address,Contact,Phone,Location,Category,Website,Email,Facebook,Twitter,Instagram,LinkedIn,Information,Genre) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?)',(name,address,contact,phone,location,category,website,email,facebook,twitter,insta,linkedin,info,genre))
                conn.commit()
                print()
                driver.back()
                a +=1
    
            
            except TimeoutException:
                pass
               
        except TimeoutException:
                    pass
    #results = driver.find_elements_by_class_name('search-result-item')
    time.sleep(0.25)
    try:
        next_page = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div/div/div/div/div[2]/div[2]/div[2]/div[1]/div/ul/li[6]/a')))
        next_page.click()
        b+=1
    except TimeoutException:
        print('TimeOut')
    


101 Collectors Records 101 West Street Main Contact - Andy Hibberd 01252 734409 Guildford RECORD SHOPS www.101collectorsrecords.co.uk andyhib101@hotmail.com  andyhib101  101andyhibberd  andyhib101 No LinkedIn Page Information
Record shop dealing in new and second hand CDs and vinyl, including most audiophile reissues. Provide mail order service. Willing to offer rack space to unsigned acts and bands, enquire in-store. Yes

3B Records 5 Slater Street Main Contact - Thomas Tuft No Phone Data Liverpool  www.3btickets.com thomas@3brecords.co.uk  3B.Records No Twitter Page No Instagram Page No LinkedIn Page Information
Vinyl and CD Dance music specialists and local ticket outlet. Stock everything from Jazz, Soul, Funk, Disco, House and Techno. Unsigned rack space available, call into the shop and speak to Thomas. Yes

81 Renshaw 81 Renshaw Street Record shop and venue space stocking all genres and providing rack space for local unsigned bands and artists. Space in-store for posters and flye

IndexError: list index out of range

In [25]:
shops = pd.read_sql_query('select * from shops',conn)

In [28]:
shops['Category'] = shops['Category'].replace('','RECORD SHOPS')

In [32]:
shops['Information'] = shops['Information'].apply(lambda x:x.split('\n')[1])

In [33]:
shops.rename({'Genre':'Unsigned Rack Space'},axis=1,inp)

,id,Name,Address,Contact,Phone,Location,Category,Website,Email,Facebook,Twitter,Instagram,LinkedIn,Information,Genre
0,1,101 Collectors Records,101 West Street,Main Contact - Andy Hibberd,01252 734409,Guildford,RECORD SHOPS,www.101collectorsrecords.co.uk,andyhib101@hotmail.com,andyhib101,101andyhibberd,andyhib101,No LinkedIn Page,Record shop dealing in new and second hand CDs...,Yes
1,2,3B Records,5 Slater Street,Main Contact - Thomas Tuft,No Phone Data,Liverpool,RECORD SHOPS,www.3btickets.com,thomas@3brecords.co.uk,3B.Records,No Twitter Page,No Instagram Page,No LinkedIn Page,Vinyl and CD Dance music specialists and local...,Yes
2,3,81 Renshaw,81 Renshaw Street,Record shop and venue space stocking all genre...,0151 707 1805,Liverpool,RECORD SHOPS,www.81renshaw.co.uk,info@81renshaw.co.uk,81Renshaw,No Twitter Page,No Instagram Page,No LinkedIn Page,Record shop and venue space stocking all genre...,Yes
3,4,Acorn Records,108a Kenwyn Street,Main Contact - Bob Dennis,01872 241368,Newquay,RECORD SHOPS,www.amazon.co.uk/shops/acorn,No Email Address,No Facebook Page,No Twitter Page,No Instagram Page,No LinkedIn Page,Deals in all genres on vinyl and CD and also s...,No
4,5,Across The Tracks,110 Gloucester Road,Main Contact - Alan Childes,01273 677906,Brighton,RECORD SHOPS,www.acrossthetracksrecords.com,No Email Address,No Facebook Page,No Twitter Page,No Instagram Page,No LinkedIn Page,"Sells most genres but specialise in Soul, Jazz...",No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,188,West Quay Records,7 West Quay,Manager - Ian,01278 428509,Taunton,RECORD SHOPS,No Website Data,No Email Address,TheNewWestQuayRecords,No Twitter Page,No Instagram Page,No LinkedIn Page,"Specialst vinyl stockists, buying and selling ...",No
188,189,White Label Records,Ivy Cottage,Main Contact - Mal White,No Phone Data,Channel Islands,RECORD SHOPS,www.discogs.com/seller/whitelabelrecords,mally@localdial.com,whitelabelrecordslimited,No Twitter Page,No Instagram Page,No LinkedIn Page,New and second hand vinyl specialists. Support...,Yes
189,190,Woodstock Vinyl,84 Brook Street,Main Contact - Brian Green,07557 647531,Chester,RECORD SHOPS,No Website Data,No Email Address,No Facebook Page,WoodstockVinyl,No Instagram Page,No LinkedIn Page,All genres catered for at Woodstock Vinyl. Rac...,Yes
190,191,Wow & Flutter,8 Trinity Street,Owner - Tim Scullion,01424 439859,Hastings,RECORD SHOPS,www.wowandflutterhastings.com,info@wowandflutterhastings.com,wowandflutterhastings,wownflutter,wowandflutterhastings,No LinkedIn Page,Independent record store with a strong focus o...,Yes


In [23]:
filename = 'Recording Studios 583 - Contacts.csv'
studi.to_csv(filename)
print('Saving File: '+filename)

Saving File: Recording Studios 583 - Contacts.csv


In [22]:
studi

,id,Name,Address,Contact,Phone,Location,Category,Website,Email,Facebook,Twitter,Instagram,LinkedIn,Information,Tools
0,1,123 Productions,Unit 4B,Producer - Brett Shaw,07703 964878,London - SE,RECORDING STUDIOS,www.123productions.co.uk,onetwothreeproductions@googlemail.com,123studios,No Twitter Page,No Instagram Page,No LinkedIn Page,"123 Studios, one of South London's top recordi...",SSE E series
1,2,2002 Studios,123a Kenton Road,Studio Manager - Steven Gurevitz,020 8907 5675,London - NW,RECORDING STUDIOS,www.2002studios.com,info@2002studios.com,No Facebook Page,No Twitter Page,No Instagram Page,No LinkedIn Page,2002 Studios is a music production company whi...,Pro Tools
2,3,2Fly Studios,Andrew Carr Farm,Owner - Alan Smyth,07725 251010,Sheffield,RECORDING STUDIOS,www.2flystudios.com,alan@2flystudios.com,pages/2Fly-Studios/124183080977234,2FlyStudio,No Instagram Page,No LinkedIn Page,2 Fly Studios is a Pro Tools studio with an ex...,Amek G2520
3,4,3rd Planet Recording Studio,17 Boundary Street,Producer - Jose Ibanez,07588 239891,Liverpool,RECORDING STUDIOS,No Website Data,3rdplanetrecording@gmail.com,3rdplanetrecordingstudios,No Twitter Page,Joseibanez,No LinkedIn Page,Control room and separated live room. Extensiv...,Hybrid analogue - Pro Tools / Logic X with ana...
4,5,45 A-Side Recordings,Unit 306,Main Contact - Paul,07803 163175,Glasgow,RECORDING STUDIOS,www.45asiderecordings.co.uk,paul@45asiderecordings.co.uk,45asiderecordings,No Twitter Page,No Instagram Page,No LinkedIn Page,The studio is centred around a 1989 ex-BBC 30 ...,Calrec C Series
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,579,Yellow Arch,30-36 Burton Road,30-36 Burton Road,0114 273 0800,Sheffield,RECORDING STUDIOS,www.yellowarch.com,Director - Andy Cook - cookinsky@gmail.com,yellowarch,yellowarch,yellowarchstudios,No LinkedIn Page,"Yellow Arch comprises a large live room, vocal...",AMEK Angela
579,580,Yellow Shark,121 Promenade,Main Contact - Danny Thomas - danny.thomas@yel...,Main Contact - Danny Thomas - danny.thomas@yel...,Cheltenham,RECORDING STUDIOS,www.yellow-shark.co.uk,Main Contact - Danny Thomas - danny.thomas@yel...,yellowsharkrecordingstudio,No Twitter Page,yellowsharkrecordingstudio,No LinkedIn Page,Established recording studio based in the cent...,Soundtracs Jade-S 48
580,581,Young Thugs Studios,Ovington Terrace,Studio Manager - J Hooker,No Phone Data,York,RECORDING STUDIOS,www.youngthugs.com/studio,jonny@youngthugs.com,youngthugsstudios,youngthugsrecs,youngthugsstudio,No LinkedIn Page,Professional recording studio consisting of 2 ...,Allen & Heath Zed 24
581,582,Zoo Audio,1d Coles Road,Proprietor - Andy Cross,01223 519296,Cambridge,RECORDING STUDIOS,www.zooaudio.co.uk,studio@zooaudio.co.uk,zooaudiouk,@zooaudio,No Instagram Page,No LinkedIn Page,Also offer location recording for live gigs. G...,Studiomaster
